In [1]:
# Copyright 2021 Xilinx, Inc.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

ChipScoPy ILA and VIO Example
=============================

Description
-----------
This example demonstrates how to program and communicate with ILA (Integrated Logic Analyzer) and
VIO (Virtual IO) cores using the ChipScoPy Python API.

<img src="img/api_overview.png" width="500" align="left">


Requirements
------------
1. Hardware Server 2021.1+
2. ChipScope Server 2021.1+
3. Xilinx Versal board such as a VCK190
4. Python 3.8 or greater with ChipScoPy 2021.1+ installed

## 1 - Initialization: Imports and File Paths

After this step,
- Required functions and classes are imported
- URL paths are set correctly
- File paths to example files are set correctly

In [2]:
import os
from chipscopy import get_examples_dir_or_die
from chipscopy import create_session, report_versions
from chipscopy.api.ila import get_waveform_data

In [3]:
# Specify locations of the running hw_server and cs_server below.
# To make things convenient, we default to values from the following environment variables.
# Modify these if needed.

CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")
BOARD = os.getenv("HW_SERVER_BOARD", "vck190/production/2.0")
EXAMPLES_DIR = get_examples_dir_or_die()
PROGRAMMING_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.pdi"
PROBES_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.ltx"

In [4]:
# Double check paths look good...
print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PROGRAMMING_FILE}")
print(f"PROBES_FILE:{PROBES_FILE}")

HW_URL: TCP:xsjmdarnall40x:3121
CS_URL: xsjmdarnall40x:3042
PROGRAMMING_FILE: C:\Users\mdarnall\dev\git\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi
PROBES_FILE:C:\Users\mdarnall\dev\git\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.ltx


## 2 - Create a session and connect to the hw_server and cs_server

Create a new session. The session is a container that keeps track of devices and debug cores managed by the hw_server and cs_server

- hw_server_url is the hardware server. It handles low level device communication
- cs_server_url is the chipscope server. It manages higher level debug core services.

After this step,

- session is initialized, pointing at a running chipscope and hardware server
- versions of ChipScoPy, hardware server, and chipscope server are known

In [5]:
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)

# Report version is a convenient command to check versions of ChipScoPy, hw_server, and cs_server.
report_versions(session)

## 3 - Program the device with our example bitstream


After this step,
- Device is programmed with the example bitstream


In [6]:
print(f"Programming {PROGRAMMING_FILE}...")
device = session.devices[0]
device.program(PROGRAMMING_FILE)

Programming C:\Users\mdarnall\dev\git\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi...


Output()

## 4 - Discover Debug Cores

Debug core discovery initializes the chipscope server debug cores. This brings debug cores in the chipscope server online.

After this step,

- chipscope server is ready for debug core use
- ila and vio core instances in the device are reported

In [7]:
device.discover_and_setup_cores(ltx_file=PROBES_FILE)
print(f"Debug cores setup and ready for use.")

Debug cores setup and ready for use.


In [8]:
# Print out the ILA core instance UUIDs and instance names
ila_cores = device.ila_cores
for index, ila_core in enumerate(ila_cores):
    print(f"{index} - {ila_core.core_info.uuid}   {ila_core.name}")

0 - 1CBBF19A124D562B8B39E5D39BB10BE3   ks_demo_i/axis_ila_dma
1 - 20E8B004FC865B87883C4CCDC77D830D   ks_demo_i/axis_ila_bram
2 - 8F904C7908B25897BEAB95475B844358   ks_demo_i/ila_fast_counter_0
3 - CA49247E0AB35CCBB6093B473F933C0E   ks_demo_i/ila_slow_counter_0


In [9]:
# Print out the VIO core instance UUIDs and instance names
vio_cores = device.vio_cores
for index, vio_core in enumerate(vio_cores):
    print(f"{index} - {vio_core.core_info.uuid}   {vio_core.name}")

0 - F77F6EBA28EE5440BA5736D4147F4955   ks_demo_i/vio_fast_counter_0
1 - 499D80541CE65035B25919B9E0CD7838   ks_demo_i/vio_slow_counter_0


## 5 - VIO Control and ILA Capture

ILA and VIO are two important building blocks for debugging applications in hardware.
This example design design shows how to control IP using a VIO core and capture results with ILA.

In this Design,
- A VIO core controls the counter (reset, up/down, ce, load)
- An ILA core captures the counter values


<img src="img/capture_data.png" width="400" align="left">

In [10]:
# Grab the two cores we are interested in for the demonstration
# As shown above, a counter is connected to the ILA core.
# The VIO core controls the counter.

ila = session.devices[0].ila_cores.get(name="ks_demo_i/ila_slow_counter_0")
vio = session.devices[0].vio_cores.get(name="ks_demo_i/vio_slow_counter_0")

print(f"Using ila: {ila.core_info.uuid}  {ila.name}")
print(f"Using vio: {vio.core_info.uuid}  {vio.name}")

Using ila: CA49247E0AB35CCBB6093B473F933C0E  ks_demo_i/ila_slow_counter_0
Using vio: 499D80541CE65035B25919B9E0CD7838  ks_demo_i/vio_slow_counter_0


### 5a - Configure the counter using VIO output probes

<img src="img/vio_control_counter.png" width="300" align="left">

In [11]:
# Set up the VIO core to enable counting up from 0
#
vio.reset_vio()
vio.write_probes(
    {
        "ks_demo_i/slow_counter_0_SCLR": 0,
        "ks_demo_i/slow_counter_0_L": 0x00000000,
        "ks_demo_i/slow_counter_0_LOAD": 0,
        "ks_demo_i/slow_counter_0_UP": 1,
        "ks_demo_i/slow_counter_0_CE": 1,
    }
)
print("Counter is now free-running and counting up")

Counter is now free-running and counting up


### 5b - Capture and display free-running counter using the ILA core

<img src="img/free_running_counter.png" width="350" align="left">

In [12]:
# Trigger ILA on the free running counter. Trigger set to the first time we see 0s in low 16-bits.
# This will show the counter is free running, and counting up
ila.reset_probes()
ila.set_probe_trigger_value("ks_demo_i/slow_counter_0_Q_1", ["==", "0xXXXX_0000"])
ila.run_basic_trigger(window_count=1, window_size=32, trigger_position=16)
print("ila is running - looking for trigger")

ila is running - looking for trigger


In [13]:
# Wait for the ila trigger with upload.
# Then print the captured ILA samples and mark the trigger position.

ila.monitor_status(max_wait_minutes=0.1)
ila.upload()
samples = get_waveform_data(
    ila.waveform,
    ["ks_demo_i/slow_counter_0_Q_1"],
    include_trigger=True,
    include_sample_info=True,
)
for trigger, sample_index, window_index, window_sample_index, value in zip(*samples.values()):
    trigger = "<-- Trigger" if trigger else ""
    print(
        f"Window:{window_index}  Window Sample:{window_sample_index}  {value:10}  0x{value:08X} {trigger}"
    )

Window:0  Window Sample:0   512622576  0x1E8DFFF0 
Window:0  Window Sample:1   512622577  0x1E8DFFF1 
Window:0  Window Sample:2   512622578  0x1E8DFFF2 
Window:0  Window Sample:3   512622579  0x1E8DFFF3 
Window:0  Window Sample:4   512622580  0x1E8DFFF4 
Window:0  Window Sample:5   512622581  0x1E8DFFF5 
Window:0  Window Sample:6   512622582  0x1E8DFFF6 
Window:0  Window Sample:7   512622583  0x1E8DFFF7 
Window:0  Window Sample:8   512622584  0x1E8DFFF8 
Window:0  Window Sample:9   512622585  0x1E8DFFF9 
Window:0  Window Sample:10   512622586  0x1E8DFFFA 
Window:0  Window Sample:11   512622587  0x1E8DFFFB 
Window:0  Window Sample:12   512622588  0x1E8DFFFC 
Window:0  Window Sample:13   512622589  0x1E8DFFFD 
Window:0  Window Sample:14   512622590  0x1E8DFFFE 
Window:0  Window Sample:15   512622591  0x1E8DFFFF 
Window:0  Window Sample:16   512622592  0x1E8E0000 <-- Trigger
Window:0  Window Sample:17   512622593  0x1E8E0001 
Window:0  Window Sample:18   512622594  0x1E8E0002 
Window:0  W

### 5c - Trigger ILA using VIO Up/Down virtual switch

This step demonstrates how VIO and ILA can be combined to form powerful debug building blocks.

ILA is set to trigger when UP/DOWN counter signal edge rises or falls.
VIO drives the UP/DOWN counter control signal to 0 causing the counter to count down.
The signal transition causes ILA to trigger and capture data.

After this step,
- VIO drives counter to count from UP to DOWN
- ILA triggers on the UP to DOWN signal transition
- Waveform uploaded with the up/down trigger sample in the center of captured data


<img src="img/edge_trigger.png" width="550" align="left">

In [14]:
# Set ILA core to capture on a transition of the UP/DOWN toggle switch
# Once transition happens, trigger in the middle of the buffer.

ila.reset_probes()
ila.set_probe_trigger_value("ks_demo_i/slow_counter_0_UP_1", ["==", "B"])
ila.run_basic_trigger(window_count=1, window_size=32, trigger_position=16)

print("ila is running - looking for trigger")

ila is running - looking for trigger


In [15]:
# VIO: Turn counter up/down switch to DOWN position.
# This will cause the running ILA to trigger on the transition edge from up to down.

vio.write_probes({"ks_demo_i/slow_counter_0_UP": 0})

print("vio changed up/down counter to count down")

vio changed up/down counter to count down


In [16]:
# Print the captured ILA samples and mark the trigger position.
# Note that counter counts down after the trigger mark.

ila.monitor_status(max_wait_minutes=0.1)
upload_successful = ila.upload()
if upload_successful:
    samples = get_waveform_data(
        ila.waveform, ["ks_demo_i/slow_counter_0_Q_1"], include_trigger=True, include_sample_info=True
    )
    for trigger, sample_index, window_index, window_sample_index, value in zip(*samples.values()):
        trigger = "<-- Trigger" if trigger else ""
        print(
            f"Window:{window_index}  Window Sample:{window_sample_index}  {value:10}  0x{value:08X} {trigger}"
        )
else:
    print("Failed to upload ILA data from core")

Window:0  Window Sample:0   564373764  0x21A3A904 
Window:0  Window Sample:1   564373765  0x21A3A905 
Window:0  Window Sample:2   564373766  0x21A3A906 
Window:0  Window Sample:3   564373767  0x21A3A907 
Window:0  Window Sample:4   564373768  0x21A3A908 
Window:0  Window Sample:5   564373769  0x21A3A909 
Window:0  Window Sample:6   564373770  0x21A3A90A 
Window:0  Window Sample:7   564373771  0x21A3A90B 
Window:0  Window Sample:8   564373772  0x21A3A90C 
Window:0  Window Sample:9   564373773  0x21A3A90D 
Window:0  Window Sample:10   564373774  0x21A3A90E 
Window:0  Window Sample:11   564373775  0x21A3A90F 
Window:0  Window Sample:12   564373776  0x21A3A910 
Window:0  Window Sample:13   564373777  0x21A3A911 
Window:0  Window Sample:14   564373778  0x21A3A912 
Window:0  Window Sample:15   564373779  0x21A3A913 
Window:0  Window Sample:16   564373780  0x21A3A914 <-- Trigger
Window:0  Window Sample:17   564373779  0x21A3A913 
Window:0  Window Sample:18   564373778  0x21A3A912 
Window:0  W